# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [2]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [3]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2:1b"

In [4]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]
print("Hai")

Hai


In [5]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [6]:
# Let's just make sure the model is loaded

!ollama pull llama3.2:1b
print("Hai")

Hai


pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest 
pulling 74701a8c

In [7]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
data = response.json()
if 'message' in data and 'content' in data['message']:
    print(data['message']['content'])
else:
    print("No content in response:", data)
print("hai")

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative models can generate high-quality, customized content such as articles, social media posts, and product descriptions. This saves time and resources for businesses that need to create content on a large scale.
2. **Virtual Reality (VR) and Augmented Reality (AR)**: Generative AI can be used to create realistic environments, characters, and objects for VR and AR experiences. This enhances the immersive experience for customers and improves product demonstration.
3. **Personalized Marketing**: By generating personalized content and experiences, businesses can tailor their marketing efforts to specific customer segments, increasing engagement and conversion rates.
4. **Chatbots and Customer Service**: Generative AI-powered chatbots can provide 24/7 customer support, answering frequent questions and helping customers with basic issues. This improves the ove

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [8]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has a wide range of business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can create high-quality, engaging content such as articles, social media posts, and product descriptions without the need for human writers or designers. This can be especially beneficial in fields like marketing, advertising, and customer service.

2. **Virtual Assistants**: AI-powered virtual assistants can generate personalized responses to customer inquiries, helping businesses provide better customer support and improving overall customer experience.

3. **Design and Product Development**: Generative AI can automate the design process for products such as packaging, product prototypes, and even entire websites. This can help companies reduce development time, improve consistency, and enhance user experience.

4. **Content Marketing**: AI-driven content generation tools can produce high-quality, relevant content at scale, freeing up huma

## Alternative approach - using OpenAI python library to connect to Ollama

In [9]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has far-reaching implications for various industries beyond its role in creative content generation. Here are some business applications:

1. **Product Design and Prototyping**: Generative AI can help designers create complex product models, prototypes, and testing environments quickly and efficiently. This leads to innovation, reduced costs, and faster time-to-market.
2. **Supply Chain Optimization**: Generative AI can predict demand, analyze data from sensors and cameras, and optimize inventory levels. This enables companies to streamline their supply chain operations, reducing waste and improving logistics efficiency.
3. **Content Creation in Advertising**: Advertisers can use generative AI to create personalized, 360-degree video commercials, product demos, or social media content that matches the user's preferences and behavior.
4. **Cybersecurity Risk Assessment**: Generative AI can analyze complex data sets and identify potential security threats based on human exp

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [10]:
!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest 
pulling aabd4debf0c8:   0% â–•                  â–� 778 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% â–•                  â–� 972 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% â–•                  â–� 1.1 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% â

In [11]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I'm trying to understand the basics behind Large Language Models (LLMs) because that's part of what I've been hearing about in AI. From what I remember, LLMs are these big models used by chatbots and other applications to generate text. But now, there's more technical underpinnings: neural networks, attention, and transformers. Let me try to break this down.

First off, neural networks. I think they're something like biological neurons, but in a machine learning context, each neuron is just a node processing inputs and outputting some value based on the weights connecting them. In neural networks for LLMs, there are layers of these nodes that process input data. Hmm, so how does this work for generating text? Maybe each layer takes the previous words or outputs from word embeddings and processes them somehow to build up the text.

Then, attention. I've heard it in contexts like machine learning where models need to focus on certain inputs more than others. Like in imag

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [14]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [15]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2:1b"

In [16]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [24]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import time
from urllib.parse import urlparse
from requests.exceptions import RequestException

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}

class Website:
    def __init__(self, url, timeout=10, delay=1):
        """
        Create this Website object from the given url using the BeautifulSoup library
        
        Args:
            url (str): The website URL to scrape
            timeout (int): Request timeout in seconds
            delay (int): Delay between requests to be polite
        """
        self.url = url
        self.timeout = timeout
        
        # Initialize all attributes with default values
        self.title = "No title found"
        self.text = "No content available"
        self.domain = None
        self.soup = None
        self.response = None
        self.metadata = {}
        
        # Be polite - add delay between requests
        time.sleep(delay)
        
        self._fetch_url()
        if self.response and self.response.status_code == 200:
            self._parse_content()
    
    def _fetch_url(self):
        """Fetch the URL with error handling"""
        try:
            self.response = requests.get(
                self.url, 
                headers=headers, 
                timeout=self.timeout,
                allow_redirects=True
            )
            self.response.raise_for_status()
            
            # Extract domain for reference
            parsed_url = urlparse(self.url)
            self.domain = parsed_url.netloc
            
        except RequestException as e:
            print(f"Error fetching {self.url}: {e}")
            self.response = None
    
    def _parse_content(self):
        """Parse the HTML content"""
        try:
            self.soup = BeautifulSoup(self.response.content, 'html.parser')
            self._extract_title()
            self._clean_content()
            self._extract_text()
            self._extract_metadata()
        except Exception as e:
            print(f"Error parsing content from {self.url}: {e}")
    
    def _extract_title(self):
        """Extract and clean the page title"""
        try:
            if self.soup.title and self.soup.title.string:
                self.title = self.soup.title.string.strip()
            else:
                # Try to find title in h1 if no title tag
                h1 = self.soup.find('h1')
                self.title = h1.get_text().strip() if h1 else "No title found"
        except Exception as e:
            print(f"Error extracting title: {e}")
            self.title = "Error extracting title"
    
    def _clean_content(self):
        """Remove irrelevant elements from the page"""
        if not self.soup or not self.soup.body:
            return
            
        try:
            # Elements to remove
            elements_to_remove = [
                "script", "style", "img", "input", "button", 
                "nav", "footer", "header", "aside", "form"
            ]
            
            for element in elements_to_remove:
                for tag in self.soup.body.find_all(element):
                    tag.decompose()
            
            # Remove comments
            for comment in self.soup.body.find_all(string=lambda text: isinstance(text, Comment)):
                comment.extract()
        except Exception as e:
            print(f"Error cleaning content: {e}")
    
    def _extract_text(self):
        """Extract and clean text content"""
        if not self.soup or not self.soup.body:
            self.text = "No content found"
            return
            
        try:
            # Get text with better formatting
            text = self.soup.body.get_text(separator="\n", strip=True)
            
            # Clean up excessive whitespace
            lines = [line.strip() for line in text.splitlines() if line.strip()]
            self.text = '\n'.join(lines) if lines else "No text content found"
        except Exception as e:
            print(f"Error extracting text: {e}")
            self.text = "Error extracting text content"
    
    def _extract_metadata(self):
        """Extract additional metadata"""
        self.metadata = {}
        
        try:
            # Meta description
            meta_desc = self.soup.find('meta', attrs={'name': 'description'})
            self.metadata['description'] = meta_desc.get('content') if meta_desc else None
            
            # Keywords
            meta_keywords = self.soup.find('meta', attrs={'name': 'keywords'})
            self.metadata['keywords'] = meta_keywords.get('content') if meta_keywords else None
            
            # Language
            html_tag = self.soup.find('html')
            self.metadata['language'] = html_tag.get('lang') if html_tag else None
        except Exception as e:
            print(f"Error extracting metadata: {e}")
    
    def get_links(self):
        """Extract all links from the page"""
        if not self.soup:
            return []
        
        links = []
        try:
            for link in self.soup.find_all('a', href=True):
                links.append({
                    'text': link.get_text(strip=True),
                    'url': link['href'],
                    'title': link.get('title', '')
                })
        except Exception as e:
            print(f"Error extracting links: {e}")
        
        return links
    
    def get_status_info(self):
        """Get HTTP status information"""
        if not self.response:
            return {"error": "No response available"}
        
        return {
            "status_code": self.response.status_code,
            "content_type": self.response.headers.get('content-type', ''),
            "content_length": len(self.response.content),
            "encoding": self.response.encoding
        }
    
    def is_successful(self):
        """Check if the website was successfully fetched and parsed"""
        return self.response is not None and self.response.status_code == 200
    
    def __str__(self):
        """String representation of the Website object"""
        return f"Website(url='{self.url}', title='{self.title}')"
    
    def __repr__(self):
        return self.__str__()

# Usage example
# if __name__ == "__main__":
#     # Example usage
#     try:
#         site = Website("https://example.com")
        
#         print(f"Title: {site.title}")
#         print(f"Domain: {site.domain}")
#         print(f"Status: {site.get_status_info()}")
#         print(f"Text length: {len(site.text)} characters")
#         print(f"Successful: {site.is_successful()}")
        
#         # Get first few links
#         links = site.get_links()[:5]
#         print("Sample links:")
#         for link in links:
#             print(f"  - {link['text']}: {link['url']}")
            
#     except Exception as e:
#         print(f"Failed to process website: {e}")



In [26]:
# Usage example
if __name__ == "__main__":
    # Example usage
    try:
        site = Website("https://www.geeksforgeeks.org/blogs/interview-preparation/")
        
        print(f"Title: {site.title}")
        print(f"Domain: {site.domain}")
        print(f"Status: {site.get_status_info()}")
        print(f"Text length: {len(site.text)} characters")
        
        # Get first few links
        links = site.get_links()[:5]
        print("Sample links:")
        for link in links:
            print(f"  - {link['text']}: {link['url']}")
            
    except Exception as e:
        print(f"Failed to process website: {e}")

Title: Interview Preparation - GeeksforGeeks
Domain: www.geeksforgeeks.org
Status: {'status_code': 200, 'content_type': 'text/html; charset="UTF-8"', 'content_length': 192202, 'encoding': 'UTF-8'}
Text length: 10481 characters
Sample links:
  - DSA: https://www.geeksforgeeks.org/dsa/dsa-tutorial-learn-data-structures-and-algorithms/
  - Practice Problems: https://www.geeksforgeeks.org/explore
  - C: https://www.geeksforgeeks.org/c/c-programming-language/
  - C++: https://www.geeksforgeeks.org/cpp/c-plus-plus/
  - Java: https://www.geeksforgeeks.org/java/java/


In [27]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [28]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    user_prompt += website.domain
    return user_prompt

In [30]:
print(user_prompt_for(site))

You are looking at a website titled Interview Preparation - GeeksforGeeks
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

DSA
Practice Problems
C
C++
Java
Python
JavaScript
Data Science
Machine Learning
Courses
Linux
DevOps
SQL
Web Development
System Design
Aptitude
Open In App
Share Your Experiences
GBlog - Explore Tech’s Hottest Topics & Career Growth Hacks!
How To Become
How to become a Java Developer?
How to Become a GenAI Developer
How to become a Cloud Network Engineer?
How to Become a DevSecOps Engineer
How to become an Automation Tester?
Roadmap
Full Stack Developer Roadmap [2025 Updated]
Complete DevOps Roadmap - Beginner to Advanced
Machine Learning Roadmap
Data Analyst Roadmap 2025 - A Complete Guide
Interview Preparation
Interview Preparation Roadmap
Top Interview Problems Asked in 2024 (Topic Wise)
Top HR Interview Questions and Answers (2025)
Database A

In [31]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [32]:
# Try this out, and then try for a few more websites

messages_for(site)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Interview Preparation - GeeksforGeeks\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nDSA\nPractice Problems\nC\nC++\nJava\nPython\nJavaScript\nData Science\nMachine Learning\nCourses\nLinux\nDevOps\nSQL\nWeb Development\nSystem Design\nAptitude\nOpen In App\nShare Your Experiences\nGBlog - Explore Tech’s Hottest Topics & Career Growth Hacks!\nHow To Become\nHow to become a Java Developer?\nHow to Become a GenAI Developer\nHow to become a Cloud Network Engineer?\nHow to Become a DevSecOps Engineer\nHow to become an Automation Tester?\nRoadmap\nFull Stack Developer Roadmap [2025 Updated]\nComplete DevOps 

In [34]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [35]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [36]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [37]:
display_summary("https://www.geeksforgeeks.org/blogs/interview-preparation/")

### GeeksforGeeks Interview Preparation - Overview
**Website Summary:**
The website provides an overview of various aspects related to computer science and interview preparation, including Programming Languages, Data Structures & Algorithms, Core Computer Science Subjects like Operating Systems, Database Management Systems, and Computer Networks. It also covers topics such as Aptitude, Reasoning, Work on Required Soft Skills, Resume Building, HR Round Questions, System Design, and courses available.

### News and Updates:
- The website has several sections dedicated to recent news and updates that can affect the industry.
- As of now, [Last Update: 26 Jul, 2025] is mentioned in the website but might be outdated. For a more current update, it's advisable to visit official sources like job portals or company announcements.

### Courses:
GeeksforGeeks offers various courses to help users prepare for their respective computer science domains.
- 
1. **Python Programming Foundation - Self Paced Course**
2. **C++ Programming Foundation - Self Paced Course**